# OPERA RTC Validation: Point Target Absolute Geolocation Evaluation

**Alex Lewandowski & Franz J Meyer; Alaska Satellite Facility, University of Alaska Fairbanks**

This notebook analyzes the absolute geolocation quality of OPERA RTC products using corner reflectors as reference. The notebook allows for analyzing corner reflector sites in California, Oklahoma, and Alaska. 

**Notebook Notes**
- Adapted for OPERA RTCs from https://github.com/OPERA-Cal-Val/calval-CSLC/blob/dev/_ALE_.ipynb

<hr>

# 0. OPERA RTC Absolute Geolocation Requirement

<div class="alert alert-success">
<i>The Sentinel-1-based RTC product (RTC-S1) shall meet an absolute geolocation accuracy better than or equal to 6 meters given the 30 meter RTC-S1 product resolution (i.e. 20% of the product resolution), excluding the effects of DEM errors, for at least 80% of all validation products considered.</i>
</div>

<hr>

# 1. Load Necessary Libraries

In [ ]:
import csv
import datetime as dt
import math
from pathlib import Path
import re
import requests
import sys
import warnings

from ipyfilechooser import FileChooser
import ipywidgets as widgets
from ipywidgets import Layout

import asf_search
import geopandas as gpd
import lmfit
from lmfit.lineshapes import gaussian2d, lorentzian
from lmfit import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyproj import Proj, CRS
import rasterio
from shapely import geometry
import shapely.wkt

from src.ALE_utils import oversample_slc, findCR
current = Path('..').resolve()
sys.path.append(str(current))
import util.geo as util

warnings.filterwarnings('ignore')

<hr>

# 2. Select OPERA RTC Products over Validation Sites

In [ ]:
print("Select the directory holding your OPERA RTCs")
fc = FileChooser(Path.cwd())
display(fc)

In [ ]:
# pass for papermill
try:
    data_dir = Path(fc.selected_path)
except:
    pass

In [ ]:
vv_path = list(Path(data_dir).rglob(f"*_30_v1.0_mosaic.tif"))[0]
s1_regex = "S1[AB]_IW_SLC__.*(?=_30_v1.0)"
try:
    s1_name = re.search(s1_regex, vv_path.stem).group(0)
except AttributeError:
    raise Exception(f"No S1 ID found in {vv_path.stem}")

print(s1_name)
    
# for Papermill
if 'savepath' not in locals():
    savepath = Path.cwd()/f"absolute_geolocation_{s1_name}"
    savepath.mkdir(exist_ok=True)
else:
    savepath = Path(savepath)

In [ ]:
dates = s1_name.split('T')[0].split('_')[5]
Year = dates[0:4]
Month = dates[4:6]
Day = dates[6:9]

Reading required file parameters

In [ ]:
with rasterio.open(vv_path) as ds:   
    start_x = ds.transform[2]+0.5*ds.transform[0]
    start_y = ds.transform[5]+0.5*ds.transform[4]
    spacing_x = ds.transform[0]
    spacing_y = ds.transform[4]
    width = ds.profile['width']
    height = ds.profile['height']
    epsg_no = ds.crs.to_epsg()
    b = ds.bounds
    

**Visualizing RTC Image**

In [ ]:
with rasterio.open(vv_path) as ds:
    rtc = ds.read(1)

# Visualize Opera Data
%matplotlib widget

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.set_title(vv_path.stem)
ax.imshow(20*np.log10(np.abs(rtc)), cmap='gray',interpolation=None, origin='upper')

<hr>

# 3. Get Corner Reflector Data and Confirm RTC Coverage

Identify which corner reflector site intersects the data

In [ ]:
ca = [-124.409591, 32.534156, -114.131211, 42.009518]
ak = [-179.148909, 51.21418, 179.77847, 71.365162]
ok = [-103.002565, 33.615833, -94.430662, 37.002206]

ll_ur_corner_coords = [ca, ak, ok]
geom = [util.poly_from_minx_miny_maxx_maxy(c) for c in ll_ur_corner_coords]
gdf = gpd.GeoDataFrame(
    {
        "dataset": ['California', 'Alaska', 'Oklahoma'],
        "geometry": geom
    }
)
gdf_4326 = gdf.set_geometry("geometry")
gdf_4326 = gdf.set_crs(f"epsg:4326")
gdf = gdf_4326.to_crs(epsg_no)

gdf = gdf.where(gdf.intersects(util.poly_from_minx_miny_maxx_maxy(b))).dropna()
gdf.reset_index(inplace=True, drop=True)
cr_zone = gdf.dataset[0]
print(f"Corner Reflector Site: {cr_zone}")

if cr_zone == 'California':
    project = 'rosamond_plate_location'
elif cr_zone == 'Alaska':
    raise Exception("Alaska CR data not currently available. Check https://uavsar.jpl.nasa.gov/cgi-bin/calibration-alaska.pl to see if status has changed.")
elif cr_zone == 'Oklahoma':
    project = 'nisar_plate_location'
else:
    raise Exception(f"{s1_name} does not intersect CA, AK, or OK corner reflector sites")

Download Corner Reflector information from UAVSAR server

In [ ]:
date_ = re.search("\d{8}T\d{6}", vv_path.stem).group(0)
date_ = dt.datetime.strptime(date_, '%Y%m%dT%H%M%S').strftime('%Y-%m-%d+%H\u0021%M')

crdata = savepath/f'{cr_zone}_{date_.split("+")[0]}_crdata.csv'
if not crdata.exists():
    res = requests.get(f'https://uavsar.jpl.nasa.gov/cgi-bin/corner-reflectors.pl?date={str(date_)}&project={project}')
    open(crdata, 'wb').write(res.content)

# Read to pandas dataframe and rename columns
df = pd.read_csv(crdata)

df.rename(columns={'   "Corner ID"':'ID'}, inplace=True)
df.rename(columns={'Latitude (deg)':'lat'}, inplace=True) 
df.rename(columns={'Longitude (deg)':'lon'}, inplace=True) 
df.rename(columns={'Azimuth (deg)':'azm'}, inplace=True)
df.rename(columns={'Height Above Ellipsoid (m)':'hgt'}, inplace=True) 
df.rename(columns={'Side Length (m)':'slen'}, inplace=True)
df.drop(columns=df.keys()[-1], inplace=True)
df

Discovering which Corner Reflectors are within RTC coverage:

In [ ]:
wkt_border = f'POLYGON(({b.left} {b.top},{b.right} {b.top},{b.right} {b.bottom},{b.left} {b.bottom},{b.left} {b.top}))'
poly = shapely.wkt.loads(wkt_border)

#calculating the locations of CRs in SAR image
UTMx = []
UTMy = []
xloc = []
yloc = []
xloc_float = []
yloc_float = []
_in = []

for idx, row in df.iterrows():
    
    _Proj = Proj(CRS.from_epsg(epsg_no))
    _x, _y = _Proj(row['lon'], row['lat'],inverse=False)     #conversion of lat/lon of CRs to UTM coordinates

    
    #location of CRs in SLC image
    _xloc = int(round((_x-start_x)/spacing_x))    
    _yloc = int(round((_y-start_y)/spacing_y))
    _
    
    UTMx.append(_x) 
    UTMy.append(_y)
    xloc.append(_xloc)
    yloc.append(_yloc)
    xloc_float.append((_x-start_x)/spacing_x)
    yloc_float.append((_y-start_y)/spacing_y)
    _in.append(poly.contains(geometry.Point(_x, _y)))
    
df['UTMx'] = UTMx
df['UTMy'] = UTMy
df['xloc'] = xloc
df['yloc'] = yloc
df['xloc_float'] = xloc_float
df['yloc_float'] = yloc_float
df['inPoly'] = _in

#checking whether CRs are in RTC coverage. Including only CRs within RTC image
df = df[df['inPoly']==True]
df.drop('inPoly', axis=1, inplace=True)
df = df.reset_index(drop=True)
df

**Visualizing** CRs on RTC Image. We color code by reflector size.

In [ ]:
%matplotlib widget
#Displaying RTC image
buffer = 50
minX = df['xloc'].min() - buffer
maxX = df['xloc'].max() + buffer
minY = df['yloc'].min() - buffer
maxY = df['yloc'].max() + buffer

scale_ = 1.0
exp_ = 0.15

fig, ax = plt.subplots(figsize=(15, 7))
cax = ax.imshow(scale_*(np.abs(rtc))**exp_, cmap='gray',interpolation='bilinear', vmin=0.3, vmax=1.7, origin='upper')
ax.set_xlim(minX,maxX)
ax.set_ylim(minY,maxY)
ax.axis('off')

for sl in pd.unique(df.slen):
    xx = df.loc[df['slen']==sl]['xloc']
    yy = df.loc[df['slen']==sl]['yloc']
    ID = df.loc[df['slen']==sl]['ID']
    
    if sl == 2.4384:
        color=[0.7, 0.7, 0.7]
    elif sl == 4.8:
        color=[0.7, 0.7, 0.7]
    elif sl == 2.8:
        color=[0.7, 0.7, 0.7]
    else:
        color=[0.7, 0.7, 0.7]
    
    ax.scatter(xx,yy,color=color,marker="o",facecolor='none',lw=1)
    for _ID,_xx,_yy in zip(ID,xx,yy):
        ax.annotate(_ID, (_xx, _yy), fontsize=10,color=[0.7, 0.7, 0.7])

ax.set_aspect(1)
plt.gca().invert_yaxis()
fig.savefig(savepath/f'{s1_name}_S1_geoRTC_CRs.png',dpi=300,bbox_inches='tight')

<hr>

# 4. Remove Corner Reflectors Facing away from the look direction of the S1 Acquisition 

In [ ]:
results = asf_search.granule_search(s1_name)
flight_direction = results[0].properties['flightDirection']
flight_direction

In [ ]:
# selecting CRs according to orbit direction
if flight_direction == 'DESCENDING':
    # descending
    df_filter = df[df['azm']>340].reset_index(drop=True)
    #only east-looking CRs (for right-looking descending)
else:
    # ascending
    df_filter = df[df['azm']<200].reset_index(drop=True)    
    #only west-looking CRs (for right-looking ascending)

df_filter = df_filter.loc[df_filter['slen']>0.8].reset_index(drop=True)   #excluding SWOT CRs (0.7 m as a side length)
df_filter

<hr>

# 5. Calculate Absolute Geolocation Error in Easting and Northing

In [ ]:
def lorentzian2d(x, y, amplitude=1., centerx=0., centery=0., sigmax=1., sigmay=1.,
                 rotation=0):
    """Return a two dimensional lorentzian.

    The maximum of the peak occurs at ``centerx`` and ``centery``
    with widths ``sigmax`` and ``sigmay`` in the x and y directions
    respectively. The peak can be rotated by choosing the value of ``rotation``
    in radians.
    """
    xp = (x - centerx)*np.cos(rotation) - (y - centery)*np.sin(rotation)
    yp = (x - centerx)*np.sin(rotation) + (y - centery)*np.cos(rotation)
    R = (xp/sigmax)**2 + (yp/sigmay)**2

    return 2*amplitude*lorentzian(R)/(np.pi*sigmax*sigmay)

def gaussfit(x, y, A, x0, y0, sigma_x, sigma_y, theta):
    theta = np.radians(theta)
    sigx2 = sigma_x**2; sigy2 = sigma_y**2
    a = np.cos(theta)**2/(2*sigx2) + np.sin(theta)**2/(2*sigy2)
    b = np.sin(theta)**2/(2*sigx2) + np.cos(theta)**2/(2*sigy2)
    c = np.sin(2*theta)/(4*sigx2) - np.sin(2*theta)/(4*sigy2)
    
    expo = -a*(x-x0)**2 - b*(y-y0)**2 - 2*c*(x-x0)*(y-y0)
    return A*np.exp(expo) 

In [ ]:
xpeak = []
ypeak = []
snr = []

ovsFactor=32

for ID, xoff, yoff in zip(df_filter['ID'],df_filter['xloc'],df_filter['yloc']):
    # crop a patch of 10*10 with center at the calculated CR position
    pxbuff = 5
    pybuff = 5
    cropcslc = rtc[(yoff-pybuff):(yoff+pybuff),(xoff-pxbuff):(xoff+pxbuff)]
    
    if np.isnan(np.mean(cropcslc))!=True:
       # _snr = get_snr_peak(cropcslc)

        # find the peak amplitude in the 10*10 patch
        yind,xind = np.unravel_index(np.argmax(np.abs(cropcslc), axis=None), cropcslc.shape)
    
        # give a warning if the peak and the calculated postion are too far
        dyind = yind-pybuff; dxind = xind-pxbuff
        dist = math.sqrt(dyind**2+dxind**2)
        if dist > 2.0:
            warnings.warn(f'the most bright pixel and the xloc is too far for CR {ID}')
    
        plt.rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(1, 3, figsize=(15, 7))
        ax[0].imshow(np.abs(cropcslc), cmap='gray',interpolation=None, origin='upper')
        ax[0].plot(xind,yind,'r+')
        ax[0].set_title(f'Corner Reflector ID: {ID}')
    
        # crop a patch of 32*32 but with its center at the peak
        xbuff = 32
        ybuff = 32
        ycrop = np.arange(yoff+dyind-ybuff,yoff+dyind+ybuff)
        xcrop = np.arange(xoff+dxind-xbuff,xoff+dxind+xbuff)
        cropcslc = rtc[ycrop,:][:,xcrop]

        # Oversample slc
        cropcslc_ovs,ycrop_ovs,xcrop_ovs = oversample_slc(cropcslc,sampling=ovsFactor,y=ycrop,x=xcrop)

        numpix = 2
    
        # find the peak amplitude again in a 2 x 2 patch, it correspond to 
        # (2*ovsFactor) x (2*ovsFactor) in oversampled slc
        yoff2 = int(cropcslc_ovs.shape[0]/2)
        xoff2 = int(cropcslc_ovs.shape[1]/2)
        cropcslc2 = cropcslc_ovs[yoff2-numpix*ovsFactor:yoff2+numpix*ovsFactor,
                               xoff2-numpix*ovsFactor:xoff2+numpix*ovsFactor]
        yind2,xind2 = np.unravel_index(np.argmax(abs(cropcslc2), axis=None), cropcslc2.shape)
        dyind2 = yind2-numpix*ovsFactor; dxind2 = xind2-numpix*ovsFactor
    
        N = numpix*2* ovsFactor
        x = np.linspace(0,numpix*2*ovsFactor-1,N)
        y = np.linspace(0,numpix*2*ovsFactor-1,N)
        Xg, Yg = np.meshgrid(x, y)
        fmodel = Model(gaussfit, independent_vars=('x','y'))
        theta = 0.1  # deg
        x0 = numpix* ovsFactor
        y0 = numpix* ovsFactor
        sigx = 2
        sigy = 5
        A = np.max(np.abs(cropcslc2))

        result = fmodel.fit(np.abs(cropcslc2), x=Xg, y=Yg, A=A, x0=x0, y0=y0, sigma_x=sigx, sigma_y=sigy, theta=theta)
        fit = fmodel.func(Xg, Yg, **result.best_values)
    
        dyind3 = result.best_values['y0']-numpix*ovsFactor; dxind3 = result.best_values['x0']-numpix*ovsFactor
    
        ax[1].imshow(np.abs(cropcslc2), cmap='gray',interpolation=None, origin='upper')
        ax[1].plot(xind2,yind2,'r+')
        ax[1].plot(result.best_values['x0'],result.best_values['y0'],'b+')
        ax[1].set_title(f'Oversampled Corner Reflector ID: {ID}')
    
        ax[2].imshow(np.abs(fit), cmap='gray',interpolation=None, origin='upper')
        ax[2].plot(xind2,yind2,'r+')
        ax[2].plot(result.best_values['x0'],result.best_values['y0'],'b+')
        ax[2].set_title(f'Oversampled Corner Reflector ID: {ID}')
    
        # crop a patch of 3x3 oversampled patch with center at the peak
        cropcslc3 = cropcslc_ovs[yoff2+dyind2-1:yoff2+dyind2+2,xoff2+dxind2-1:xoff2+dxind2+2]
        ycrop2 = ycrop_ovs[yoff2+dyind2-1:yoff2+dyind2+2]
        xcrop2 = xcrop_ovs[xoff2+dxind2-1:xoff2+dxind2+2]
        xxcrop2,yycrop2 = np.meshgrid(xcrop2,ycrop2)
        xxcrop2_f = xxcrop2.flatten()
        yycrop2_f = yycrop2.flatten()
        cropcslc2_f = cropcslc3.flatten()

        # Check if pixel values in a patch are non-NaN
        valid = ~(np.isnan(cropcslc2_f))
        count_valid = np.count_nonzero(valid)

        if count_valid == 0:
            _ypeak, _xpeak = [np.nan, np.nan]

        else:
            _ypeak,_xpeak = findCR(np.abs(cropcslc2_f[valid]),yycrop2_f[valid],xxcrop2_f[valid],
                                x_bound=[xcrop2[0],xcrop2[-1]],y_bound=[ycrop2[0],ycrop2[-1]],method="para")

        #xpeak.append(_xpeak)
        #ypeak.append(_ypeak)
        #xpeak.append(xoff+dxind+dxind2/ovsFactor)
        #ypeak.append(yoff+dyind+dyind2/ovsFactor)
        xpeak.append(xoff+dxind+dxind3/ovsFactor)
        ypeak.append(yoff+dyind+dyind3/ovsFactor)
        #snr.append(_snr)
    else:
        xpeak.append(np.nan)
        ypeak.append(np.nan)
    
df_filter['xloc_CR'] = xpeak
df_filter['yloc_CR'] = ypeak
#df_filter['snr'] = snr
df_filter

**Visualizing CR Location Measurements**

In [ ]:
df_filter = df_filter.dropna().reset_index(drop=True)
df_filter

## 5.1 Uncomment this Line to Drop CRs that were Poorly Identified

In [ ]:
# df_filter = df_filter.drop([2]).reset_index(drop=True)

## 5.2 Calculating Absolute Geolocation Numbers:

In [ ]:
#absloute geolocation error in range and azimuth
ALE_Rg = (df_filter['xloc_CR'] -  df_filter['xloc_float']) * spacing_x
ALE_Az = (df_filter['yloc_CR'] - df_filter['yloc_float']) * spacing_y

#test_Rg = ((df_filter['xloc_float'] % 1.0)-0.5)
#test_Az = -((df_filter['yloc_float'] % 1.0)-0.5)

test_Rg = ((df_filter['xloc']-df_filter['xloc_float']))
test_Az = -((df_filter['yloc']-df_filter['yloc_float']))

In [ ]:
ALE_Az

In [ ]:
ALE_Rg

In [ ]:
test_Az

In [ ]:
np.sqrt(test_Az**2 + test_Rg**2)

<hr>

## 5.3 Removing Corner Reflectors that are near the Edge of the Pixel


As we cannot interpolate into the pixel, corner reflectors that sit near the edge of the pixel can bias our offset estimate. Therefore, we are removing corner reflectors near pixel edges before we analyze summary statistics. 

In [ ]:
subpix = np.sqrt(test_Az**2 + test_Rg**2)
keepind = []
for idx, row in subpix.items():
    if subpix[idx] <=0.55:
        keepind.append(idx)
print(keepind)

# 6. Plot Absolute Geolocation Error in Easting and Northing

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
requirement = plt.Rectangle((-3.0,-3.0), 6.0, 6.0, fill=False, edgecolor='grey', label='Requirement')
ax.add_patch(requirement)
#sc = ax.scatter(ALE_Rg, ALE_Az, s=200, c=df_filter['slen'], alpha=0.6, marker='o')
sc = ax.scatter(ALE_Rg[keepind], ALE_Az[keepind], s=100, c='k', alpha=0.6, marker='o')
#ax.legend(*sc.legend_elements(),facecolor='lightgray')
#ax.get_legend().set_title('side length (m)')


for ii, txt in enumerate(df_filter.iloc[keepind,0]):
    ax.annotate(txt, (ALE_Rg[keepind[ii]],ALE_Az[keepind[ii]]), color='black',xytext=(0, 5), textcoords='offset points')   #putting IDs in each CR
    
ax.grid(True)
ax.set_xlim(-15.25,15.25)
ax.set_ylim(-15.25,15.25)
ax.axhline(0, color='black')
ax.axvline(0, color='black')

#np.std(data, ddof=1) / np.sqrt(np.size(data))

ax.set_title(f'Easting: {np.round(np.nanmean(ALE_Rg[keepind]), 3)} +/- {np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3)} m, \
    Northing: {np.round(np.nanmean(ALE_Az[keepind]),3)}, +/- {np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3)} m')
ax.set_xlabel('Easting error (m)')
ax.set_ylabel('Northing error (m)')
fig.suptitle('Absolute Geolocation Error')

plt.errorbar(np.round(np.nanmean(ALE_Rg[keepind]), 3), np.round(np.nanmean(ALE_Az[keepind]),3),\
             xerr=np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3), yerr=np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3), \
             barsabove=True, capsize=8, capthick=2, fmt='ro', linewidth=2, markersize=20)

output = f"{s1_name}_GeolocationPLOT.png"
plt.savefig(savepath/output, dpi=300, transparent='true')

In [ ]:
print(s1_name)
print(np.round(np.nanmean(ALE_Rg[keepind]), 3))
print(np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3))
print(np.round(np.nanmean(ALE_Az[keepind]),3))
print(np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3))

In [ ]:
#plotting ALE

#msize = (df_filter['CRZscrores'] - np.min(df_filter['CRZscrores']) + 0.000001) * 100.0

fig, ax = plt.subplots(figsize=(8,8))
requirement = plt.Rectangle((-3.0,-3.0), 6.0, 6.0, fill=False, edgecolor='grey', label='Requirement')
ax.add_patch(requirement)
#sc = ax.scatter(ALE_Rg, ALE_Az, s=200, c=df_filter['slen'], alpha=0.6, marker='o')
sc = ax.scatter(test_Rg, test_Az, s=100, c='k', alpha=0.6, marker='o')


for ii, txt in enumerate(df_filter.iloc[:,0]):
    ax.annotate(txt, (test_Rg[ii],test_Az[ii]), color='black',xytext=(0, 5), textcoords='offset points')   #putting IDs in each CR
    
ax.grid(True)
ax.set_xlim(-1.25,1.25)
ax.set_ylim(-1.25,1.25)
ax.axhline(0, color='black')
ax.axvline(0, color='black')

ax.set_title(f'Easting: {np.round(np.nanmean(test_Rg), 3)} +/- {np.round(np.nanstd(test_Rg) / np.sqrt(np.size(test_Rg)),3)} m, \
    Northing: {np.round(np.nanmean(test_Az),3)}, +/- {np.round(np.nanstd(test_Az) / np.sqrt(np.size(test_Az)),3)} m')
ax.set_xlabel('Easting error (m)')
ax.set_ylabel('Northing error (m)')
fig.suptitle('Fractional Offset from Pixel Center')

plt.errorbar(np.round(np.nanmean(test_Rg), 3), np.round(np.nanmean(test_Az),3),\
             xerr=np.round(np.nanstd(test_Rg) / np.sqrt(np.size(test_Rg)),3), yerr=np.round(np.nanstd(test_Az) / np.sqrt(np.size(test_Az)),3), \
             barsabove=True, capsize=8, capthick=2, fmt='ro', linewidth=2, markersize=20)

output = f"{s1_name}_FracOffset.png"
plt.savefig(savepath/output, dpi=300, transparent='true')

# 7. Write Results into a CSV File

In [ ]:
ALE_csv = savepath.parent/f"{cr_zone}_ALE30-Results.csv"

fields = ["Granule", "Year", "Month", "Day", "Easting_Bias", "sig_Easting_Bias", "Northing_Bias", "sig_Northing_Bias"]

row = [s1_name, Year, Month, Day,  
       np.round(np.nanmean(ALE_Rg[keepind]), 3), np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3),
       np.round(np.nanmean(ALE_Az[keepind]), 3), np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3)]

if not ALE_csv.exists():
    with open(ALE_csv, 'w') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)
        csvwriter.writerow(row)
else:
    with open(ALE_csv, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        s1_names = [c[0] for c in list(csvreader)]
    if s1_name not in s1_names:
        with open(ALE_csv, 'a') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row)

In [ ]:
ALE_csv

*ALE_OPERA_RTC.ipynb - Version 2.0.0 - April 2023*

*Change log*

- Made CR discovery more robust
- Added average visualization in geolocation plot
- Made formatting changes